In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 4 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Analyse sur la stabilité des algorithmes de résolution directe de système linéaires
#         Choix du pivotage ou non, pivot partiel ou total.
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

# Choix du pivot         

In [ ]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpmath import mp

from scipy.stats import ortho_group

In [ ]:
def gaussian_elimination_without_pivoting(a, b): 
    n = b.size
    for i in range(n-1):
        # elimination
        li = a[i+1:,i]/a[i,i]
        b[i+1:] = b[i+1:] - li * b[i]
        a[i+1:] = a[i+1:] - li.reshape(n-i-1,1)*a[i]

def gaussian_elimination_with_partial_pivoting(a, b): 
    n = b.size
    for i in range(n-1):
        # partial pivoting    
        i_max = np.argmax(np.abs(a[i:,i]))
        a[[i,i_max+i]] = a[[i_max+i,i]]
        b[[i,i_max+i]] = b[[i_max+i,i]]
        # elimination
        li = a[i+1:,i]/a[i,i]
        b[i+1:] = b[i+1:] - li * b[i]
        a[i+1:] = a[i+1:] - li.reshape(n-i-1,1)*a[i]
        
def gaussian_elimination_with_total_pivoting(a, b): 

    n = b.size
    ord_col = np.arange(n)
    for i in range(n-1):
        # total pivoting
        i_max, j_max = np.unravel_index(np.abs(a[i:,i:]).argmax(), a[i:,i:].shape)
        a[[i,i_max+i]] = a[[i_max+i,i]]
        a[:,[i,j_max+i]] = a[:,[j_max+i,i]]
        b[[i,i_max+i]] = b[[i_max+i,i]]
        ord_col[[i,j_max+i]] = ord_col[[j_max+i,i]]
        # elimination
        li = a[i+1:,i]/a[i,i]
        b[i+1:] = b[i+1:] - li * b[i]
        a[i+1:] = a[i+1:] - li.reshape(n-i-1,1)*a[i]
        
    return ord_col

def backward_substitution(a, b):
    n = b.size
    x = np.empty(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.sum(a[i,i+1:]*x[i+1:])) / a[i,i]
    return x

def gauss_solve_without_pivoting(a, b):
    ag = np.copy(a) 
    bg = np.copy(b)
    gaussian_elimination_without_pivoting(ag, bg)
    return backward_substitution(ag, bg)

def gauss_solve_with_partial_pivoting(a, b):
    ag = np.copy(a) 
    bg = np.copy(b)
    gaussian_elimination_with_partial_pivoting(ag, bg)
    return backward_substitution(ag, bg)


def gauss_solve_with_total_pivoting(a, b):
    ag = np.copy(a) 
    bg = np.copy(b)
    ord_col = gaussian_elimination_with_total_pivoting(ag, bg)
    x = backward_substitution(ag, bg)
    return x[ord_col.argsort()]

## Matrices aléatoires

Pour chaque taille n = 5,6,...,55 nous choisissons 80 matrices aléatoires avec coefficients $a_{ij}$ uniformément distribués dans $[−1, 1]$ et des solutions $x_i$ uniformément distribuées dans $[−1, 1]$ en double précision. Nous calculons alors en quadruple précision les $b_j$ pour cette solution exacte. Ensuite nous appliquons l’algorithme de Gauss, une fois sans recherche de pivot, puis avec recherche de pivot partielle et avec recherche de pivot totale, en double précision. 

Le fait d'avoir accès à la solution exacte et à une solution en double précision pour les divers algorithmes permet une évaluation de l'erreur générée par l'utilisation de l'algorithme en double précision.

In [ ]:
def test_gauss_without_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))

    print("\nGauss without pivoting")

    for i, i_n in enumerate(range(5,56)):
        for j in range(n_test):
            a = mp.mpf('2.0')*np.random.random((i_n,i_n))-1
            # à décommenter pour voir l'impact du pivot total
            # a[1,i_n-1] = mp.mpf('100000')
            x_ex = mp.mpf('2.0')*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_without_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

def test_gauss_with_partial_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))
        
    print("\nGauss with partial pivoting")
 
    for i, i_n in enumerate(range(5,56)):
        #print(".", end="")
        for j in range(n_test):
            a = mp.mpf('2.0')*np.random.random((i_n,i_n))-1
            # à décommenter pour voir l'impact du pivot total
            # a[1,i_n-1] = mp.mpf('100000')
            x_ex = mp.mpf('2.0')*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_with_partial_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

def test_gauss_with_total_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))
        
    print("\nGauss with total pivoting")
 
    for i, i_n in enumerate(range(5,56)):
        for j in range(n_test):
            a = mp.mpf('2.0')*np.random.random((i_n,i_n))-1
            # à décommenter pour voir l'impact du pivot total
            # a[1,i_n-1] = mp.mpf('100000')
            x_ex = mp.mpf('2.0')*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_with_total_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

In [ ]:
# nb de matrices testées par taille
n_test = 80

err_1, errovercond_1 = test_gauss_without_pivoting(n_test)
err_2, errovercond_2 = test_gauss_with_partial_pivoting(n_test)
err_3, errovercond_3 = test_gauss_with_total_pivoting(n_test)

La norme infinie de l'erreur $(x_i^{num} − x_i^{ex})$ puis la norme infinie de l'erreur divisée par le conditionnement de chaque résultat est représentée sur la figure suivante :

In [ ]:
fig = make_subplots(rows=2, cols=3, vertical_spacing=0.1, subplot_titles=("|err|_inf", "|err|_inf", "|err|_inf", "|err|_inf/cond", "|err|_inf/cond", "|err|_inf/cond>"))

for i, i_n in enumerate(range(5,56)):
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_1[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_2[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=2)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_3[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=3)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_1[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_2[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=2)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_3[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=3)

fig.update_yaxes(type="log", range=[-17,-8],  exponentformat = 'e', row=1)    
fig.update_yaxes(type="log", range=[-19,-11], exponentformat = 'e', row=2)    
    
fig.update_layout(height=1000)

fig.show()

Dans le cas sans recherche de pivot, le niveau général d'erreur est bien supérieur à celui du cas avec recherche de pivot. Le conditionnement est le même dans les deux cas, celui de
la matrice $A$.

Afin de préciser l'origine des erreurs, nous avons représenté l'erreur précédente divisée par le conditionnement de la matrice. Il apparaît clairement que ce nombre reste en dessous de la précision machine en double précision dans le cas où l'on emploie une technique de pivotage. Cela indique un algorithme très stable au sens backward. Ce n'est pas la cas de l'algorithme sans pivot qui peut poser de sérieux problèmes de précision. Le choix du pivotage partiel mène à une algorithme qui se comporte bien dans la plupart des cas mais à une coût moindre par rapport au pivot total que l'on utile plus rarement.

## Matrices orthogonales

Pour cette deuxième expérience numérique, nous tirons 80 matrices aléatoires $a_{ij}$ orthogonale . Cette matrice est calculée en double précision, le reste de l’expérience continue comme auparavant.

In [ ]:
def test_ortho_gauss_without_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))

    print("\nGauss without pivoting")

    for i, i_n in enumerate(range(5,56)):
        for j in range(n_test):
            a = mp.mpf('1.0')*ortho_group.rvs(dim=i_n)
            x_ex = mp.mpf('2.0')*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_without_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

def test_ortho_gauss_with_partial_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))
        
    print("\nGauss with partial pivoting")
 
    for i, i_n in enumerate(range(5,56)):
        for j in range(n_test):
            a = mp.mpf('1.0')*ortho_group.rvs(dim=i_n)
            x_ex = 2*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_with_partial_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

def test_ortho_gauss_with_total_pivoting(n_test):

    err = np.zeros((51,n_test))
    errovercond = np.zeros((51,n_test))
        
    print("\nGauss with total pivoting")
 
    for i, i_n in enumerate(range(5,56)):
        for j in range(n_test):
            a = mp.mpf('1.0')*ortho_group.rvs(dim=i_n)
            x_ex = 2*np.random.random(i_n)-1
            b = np.dot(a, x_ex)
            a_64 = a.astype(np.float64)
            b_64 = b.astype(np.float64)
            x_num = gauss_solve_with_total_pivoting(a_64, b_64)
            err[i,j] = np.linalg.norm(np.abs(x_num-x_ex), np.inf)
            errovercond[i,j] = err[i,j] / np.linalg.cond(a_64, np.inf)
            
    return err, errovercond

In [ ]:
# nb de matrices testées par taille
n_test = 80

err_4, errovercond_4 = test_ortho_gauss_without_pivoting(n_test)
err_5, errovercond_5 = test_ortho_gauss_with_partial_pivoting(n_test)
err_6, errovercond_6 = test_ortho_gauss_with_total_pivoting(n_test)

La norme infinie de l'erreur $(x_i^{num} − x_i^{ex})$ puis la norme infinie de l'erreur divisée par le conditionnement de chaque résultat est représentée sur la figure suivante :

In [ ]:
fig = make_subplots(rows=2, cols=3, vertical_spacing=0.1, subplot_titles=("|err|_inf", "|err|_inf", "|err|_inf", "|err|_inf/cond", "|err|_inf/cond", "|err|_inf/cond>"))

for i, i_n in enumerate(range(5,56)):
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_4[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_5[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=2)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=err_6[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=1, col=3)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_4[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_5[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=2)
    fig.add_trace(go.Scatter(x=np.ones(n_test)*i_n, y=errovercond_6[i], showlegend=False, mode="markers", marker_color='blue', marker_size=3), row=2, col=3)

fig.update_yaxes(type="log", range=[-17,-8],  exponentformat = 'e', row=1)    
fig.update_yaxes(type="log", range=[-18,-10], exponentformat = 'e', row=2)    
    
fig.update_layout(height=1000)

fig.show()

Nous constatons que lorsque nous utilisons la recherche de pivot partielle,  il n'y a pas d'exception dans la bonne performance de l'algorithme de Gauss puisqu'on ne peut pas avoir de matrice mal-conditionnée dans ce cas! 

Par ailleurs, les conclusions sont les mêmes en ce qui concerne la stabilité des algorithmes, à ceci près que les erreurs ici sont uniquement générées par la perte potentielle de stabilité des algorithmes, puisque toutes les matrices sont bien conditionnées. 

Remarquons que ce sont les erreurs en norme infinie qui sont présentées et que l'on observe une croissance légère avec la dimension des matrices, ce qui ne serait pas le cas en norme 2.
